In [2]:
import pandas as pd
import plotly.express as px

In [3]:
raw_data = pd.read_csv('grade.csv')

In [4]:
raw_data.columns

Index(['First name', 'Surname', 'ID number', 'Institution', 'Department',
       'Email address', 'Quiz: Written test: 1st attempt (Real)',
       'Quiz: Written test: 1st attempt (Letter)',
       'Quiz: Written test: 2nd attempt (Real)',
       'Quiz: Written test: 2nd attempt (Letter)',
       'Quiz: Written test: 3rd attempt (Real)',
       'Quiz: Written test: 3rd attempt (Letter)', 'Course total (Real)',
       'Course total (Letter)', 'Last downloaded from this course'],
      dtype='object')

In [5]:
subset_data = raw_data.drop(columns=raw_data.columns[[3,4,7,9,11,14]])
subset_data.columns = ['firstname','lastname','id','email','test_1','test_2','test_3','course_total','grade']
# Replace missing grade with E
subset_data['course_total'].replace('-',0,inplace=True)
subset_data['grade'].replace('-','E',inplace=True)
# Get max score from all tests
subset_data['course_grade_check'] = subset_data[['test_1','test_2','test_3']].max(axis=1)
# Coerce data to number and check if manual calculation is the same as LMS?
subset_data['course_grade_check'] = pd.to_numeric(subset_data['course_grade_check'],errors='coerce').fillna(0)*2
subset_data['course_total'] = pd.to_numeric(subset_data['course_total'],errors='coerce').fillna(0)
check_data = (subset_data['course_total'] == subset_data['course_grade_check'])

In [6]:
check_data = (subset_data['course_total'] == subset_data['course_grade_check'])
if check_data.all():
    print("Manual calculation agrees with LMS calculation")
else:
    raise Exception("Manual calculation disagreed with LMS calculation")

Manual calculation agrees with LMS calculation


In [7]:
def grade_cut(single_score):
    if single_score >= 89.5:
        return 'A'
    elif single_score >=84.5 and single_score < 89.5:
        return 'B+'
    elif single_score >= 79.5 and single_score < 84.5:
        return 'B'
    elif single_score >= 74.5 and single_score < 79.5:
        return 'C+'
    elif single_score >= 69.5 and single_score < 74.5:
        return 'C'
    elif single_score >= 59.5 and single_score < 69.5:
        return 'D+'
    elif single_score >= 49.5 and single_score < 59.5:
        return 'D'
    else:
        return 'E'
    
def grade_students(total_score):
    return [grade_cut(x) for x in total_score]

In [8]:
subset_data['manual_grade'] = grade_students(subset_data['course_total'])

In [9]:
subset_data['grade']

0      D
1      A
2      A
3      E
4     B+
      ..
64    D+
65     B
66     C
67     D
68     D
Name: grade, Length: 69, dtype: object

In [10]:
grade_check = (subset_data['grade'] == subset_data['manual_grade'])
if grade_check.all():
    print("Grade check successful. All calculation completed.")
else:
    raise Exception("grade check failed. Manual cal doesn't agree.")

Grade check successful. All calculation completed.


In [11]:
subset_data.to_excel('check.xlsx')

In [12]:
grade_plot = px.histogram(subset_data,x='grade',category_orders={'grade':['A','B+','B','C+','C','D+','D','E']})

In [18]:
grade_plot.write_image('grade_hist.png')